# BO runs

In [1]:
import torch
from botorch.models import FixedNoiseGP, SingleTaskGP
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.analytic import ExpectedImprovement
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys
import time

In [2]:
which_acquisition = "EI"
# which_acquisition = "max y_hat"
# which_acquisition = "max sigma"

load data from `prepare_Xy.ipynb`

In [3]:
X = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['X']
y = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['y']
y = np.reshape(y, (np.size(y), 1)) # for the GP
nb_data = np.size(y)
nb_data

69839

convert to torch tensors

In [4]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [5]:
X.size()

torch.Size([69839, 12])

In [6]:
y.size()

torch.Size([69839, 1])

In [7]:
X_unsqueezed = X.unsqueeze(1)

In [8]:
nb_data

69839

number of COFs for initialization

In [9]:
nb_COFs_initialization = 10

In [27]:
def bo_run(nb_iterations, verbose=False):
    assert nb_iterations > nb_COFs_initialization
    
    # select initial COFs for training data randomly.
    # idea is to keep populating this ids_acquired and return it for analysis.
    ids_acquired = np.random.choice(np.arange((nb_data)), size=nb_COFs_initialization, replace=False)

    # initialize acquired y, since it requires normalization
    y_acquired = y[ids_acquired]
    # standardize outputs
    y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
    
    for i in range(nb_COFs_initialization, nb_iterations):
        print("iteration:", i, end="\r")
        # construct and fit GP model
        model = SingleTaskGP(X[ids_acquired, :], y_acquired)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_model(mll)

        # set up acquisition function
        if which_acquisition == "EI":
            acquisition_function = ExpectedImprovement(model, best_f=y_acquired.max().item())
            
            # compute aquisition function at each COF in the database. need to do in batches to avoid mem issues
            batch_size = 35000
            acquisition_values = torch.zeros((nb_data))
            acquisition_values[:] = np.NaN # for safety
            nb_batches = nb_data // batch_size
            for ba in range(nb_batches+1):
                id_start = ba * batch_size
                id_end   = id_start + batch_size
                if id_end > nb_data:
                    id_end = nb_data
                acquisition_values[id_start:id_end] = acquisition_function.forward(X_unsqueezed[id_start:id_end])
            assert acquisition_values.isnan().sum().item() == 0 # so that all are filled properly.
            del acquisition_function
    #         acquisition_values = acquisition_function.forward(X_unsqueezed) # runs out of memory
        elif which_acquisition == "max y_hat":
            acquisition_values = model.posterior(X_unsqueezed).mean.squeeze()
        elif which_acquisition == "max sigma":
            acquisition_values = model.posterior(X_unsqueezed).variance.squeeze()
        else:
            raise Exception("not a valid acquisition function")

        # select COF to acquire with maximal aquisition value, which is not in the acquired set already
        ids_sorted_by_aquisition = acquisition_values.argsort(descending=True)
        for id_max_aquisition_all in ids_sorted_by_aquisition:
            if not id_max_aquisition_all.item() in ids_acquired:
                id_max_aquisition = id_max_aquisition_all.item()
                break

        # acquire this COF
        ids_acquired = np.concatenate((ids_acquired, [id_max_aquisition]))
        assert np.size(ids_acquired) == i + 1

        # update y aquired; start over to normalize properly
        del y_acquired
        y_acquired = y[ids_acquired, :] # start over to normalize y properly
        y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
        
        if verbose:
            print("\tacquired COF", id_max_aquisition, "with y = ", y[id_max_aquisition].item())
            print("\tbest y acquired:", y[ids_acquired].max().item())
        
        del model
        del mll
        del acquisition_values
        
    assert np.size(ids_acquired) == nb_iterations
    return ids_acquired

`ids_acquired[r, i]` will give ID of COF acquired during iteration `i` from run `r`.

In [ ]:
bo_res = dict()
bo_res['nb_runs']       = 50
bo_res['nb_iterations'] = 250
bo_res['ids_acquired'] = []
for r in range(bo_res['nb_runs']):
    print("\n\nRUN", r)
    t0 = time.time()
    ids_acquired = bo_run(bo_res['nb_iterations'])
    bo_res['ids_acquired'].append(ids_acquired)
    print("took time t = ", (time.time() - t0) / 60, "min")



RUN 0


In [25]:
with open('bo_results' + which_acquisition + '.pkl', 'wb') as file:
    pickle.dump(bo_res, file)